In [0]:
#packages

import requests
import pandas as pd
import datetime
from datetime import date

In [0]:
#creation of date[] list which stores date for upcoming 8 days

today = date.today()
start_date = datetime.date(today.year,today.month,today.day)
number_of_days = 8
date_list=[]
for day in range(number_of_days):
    a_date = (start_date + datetime.timedelta(days = day)).isoformat()
    date_list.append(a_date)

In [0]:
#Accesing weather data using openWeatherMapApi

api_key = 'db36763c5f342f29ac7e24db3badc4a9'

#Creating dataframe
df = pd.DataFrame(columns = ['Date', 'Day_temperature', 'Night_temperature','Weather_Condition','Humidity','Wind_Speed','State'])

#State_List which stores all 29 states
state_list=['andhra pradesh','Arunachal Pradesh','Assam','Haryana','Karnataka','Kerala','Goa','Gujarat','Himachal Pradesh','Jammu and Kashmir','Madhya Pradesh','Maharashtra','Manipur','Mizoram','Nagaland','New Delhi','odisha','punjab','Rajasthan','sikkim','tamil nadu','Hyderabad','Tripura','Uttarakhand','Uttar pradesh','west bengal','Bihar','jharkhand','meghalaya']

for state in state_list:
    state_name=state
    
    #Gives latitude and longitude values for particular state
    url = f'https://api.openweathermap.org/data/2.5/weather?q={state_name}&appid={api_key}'
    req = requests.get(url)
    data = req.json()
    lon = data['coord']['lon']
    lat = data['coord']['lat']
    
    #Gives data for 8days, excluding minute and hour data
    exclude = "minute,hourly"
    data_url = f'https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude={exclude}&appid={api_key}'
    data_req = requests.get(data_url)
    eight_days_data = data_req.json()
    
    
    #Accessing data from json file recived from API and storing it in pandas dataframe
    j=0
    for i in eight_days_data['daily']:
        df=df.append({'Date':date_list[j],'Day_temperature' : round(i['temp']['day'] - 273.15,2), 'Night_temperature' : round(i['temp']['night'] - 273.15,2), 'Weather_Condition' : i['weather'][0]['main'] + ": " +i['weather'][0]['description'],'Humidity':i['humidity'],'Wind_Speed':i['wind_speed'],'State':state_name.title()}, ignore_index = True)
        j=j+1


In [0]:
#Converting pandas dataframe to spark dataframe 

final_df = spark.createDataFrame(df)
final_df.show(10)

+----------+---------------+-----------------+--------------------+--------+----------+-----------------+
 Date|Day_temperature|Night_temperature| Weather_Condition|Humidity|Wind_Speed| State|
+----------+---------------+-----------------+--------------------+--------+----------+-----------------+
2022-04-21| 36.76| 24.14| Clear: clear sky| 21| 6.76| Andhra Pradesh|
2022-04-22| 34.99| 23.87|Clouds: broken cl...| 22| 4.78| Andhra Pradesh|
2022-04-23| 37.52| 26.09|Clouds: scattered...| 18| 4.83| Andhra Pradesh|
2022-04-24| 38.58| 27.13| Clouds: few clouds| 20| 2.32| Andhra Pradesh|
2022-04-25| 39.95| 27.53| Rain: light rain| 19| 4.55| Andhra Pradesh|
2022-04-26| 39.04| 26.99|Clouds: overcast ...| 21| 4.37| Andhra Pradesh|
2022-04-27| 39.52| 27.25| Clear: clear sky| 21| 4.51| Andhra Pradesh|
2022-04-28| 41.02| 27.53| Rain: light rain| 18| 7.67| Andhra Pradesh|
2022-04-21| 28.58| 18.2| Rain: moderate rain| 45| 2.5|Arunachal Pradesh|
2022-04-22| 23.39| 18.2| Rain: moderate rain| 71| 1.46|Arunachal Pradesh|
+----------+---------------+-----------------+--------------------+--------+----------+-----------------+
only showing top 10 rows

In [0]:
#Saving the dataframe as table

final_df.write.mode("overwrite").saveAsTable("default.PREDICTED_WEATHER_DATA")